In [ ]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import re

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
def data_load():
    file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
    train_data = pd.read_csv(file_path+"train_data_pre1_0820.csv")
    cr_data = pd.read_csv(file_path+"crawl_uniquecode(0825).csv")
    return train_data, cr_data

In [ ]:
train_data , cr_data = data_load()

In [ ]:
# make dictionary code to product name 
code2name_dict = dict(zip(train_data['p_code'].to_list(), train_data['p_name'].to_list()))

2124

In [ ]:
def code2name(code):
    return code2name_dict[code]

In [ ]:
code2name_dict[201220]

'그렉노먼 남성 구스다운 롱 벤치코트'

In [ ]:
def crawl(keyword):
    url = f"https://search.shopping.naver.com/search/all.nhn?query={keyword}&cat_id=&frm=NVSHATC"
    data = requests.get(url)
    return data.content

In [ ]:
def getCrawlInfo(li):
    # get Productname, link, price 
    aTit = li.find("a", {"class": "basicList_link__1MaTN"})
    name = aTit.text
    link = aTit['href']
    price = li.find("span", {"class":"price_num__2WUXn"}).text.replace(",", "")

    # get Categories - 대/ 중/ 소  (없으면 None 으로 표기되도록)
    cate_class = li.findAll("a", {"class": "basicList_category__wVevj"})
    cate_list = [None, None, None]
    for idx in range(3):
        if cate_class[idx] :
            cate_list[idx] = cate_class[idx].text
        else:
            pass

    result_dict = {"name":name, "link": link, "price":price, "cate1":cate_list[0] , "cate2":cate_list[1], "cate3":cate_list[2]}
    return result_dict

In [ ]:
def parse(pageString):
    bsObj = BeautifulSoup(pageString, "html.parser")
    ul = bsObj.find("ul", {"class":"list_basis"})

    if ul: # 검색 결과 있는 경우 
        lis = ul.find("li",{"class":"basicList_item__2XT81"}) 
        crawl_info_dict = getCrawlInfo(lis)
        crawl_info_dict['cr'] = 1 # 검색 결과 여부도 labeling - (결과 있으면 1)

    else: # 검색 결과 없는 경우 
        key_list = ["name", "link", "price", "cate1", "cate2", "cate3"]
        crawl_info_dict = dict(zip(key_list, [None]*6))
        crawl_info_dict['cr'] = 0  # 검색 결과 여부도 labeling - (결과 없으면 0)

    return crawl_info_dict

In [ ]:
def get_crawl_data(p_code, p_name):
    try : 
        pageString = crawl(p_name)
        crawl_info_dict = parse(pageString)
        crawl_info_dict['p_code'] = p_code
    except: 
        key_list = ["name", "link", "price", "cate1", "cate2", "cate3"]
        crawl_info_dict = dict(zip(key_list, [None]*6))
        crawl_info_dict['cr'] = -1  #에러 결과 여부도 labeling -1
        crawl_info_dict['p_code'] = p_code
    return crawl_info_dict

In [ ]:
def main(code2name_dict):

    all_dict_list = []
    for p_code, p_name in tqdm(code2name_dict.items()):
        print()
        one_crawl_dict = get_crawl_data(p_code, p_name)
        all_dict_list.append(one_crawl_dict)
        print(p_code, p_name,one_crawl_dict )
        
    return all_dict_list


In [ ]:
## CRAWLING ! 
all_dict_list = main(code2name_dict)

In [ ]:
import json
file_path = "/content/drive/My Drive/Colab Notebooks/2020bigcontest/data/"
result_file = open(file_path+"result_dict(0825).json",'w+')
result_file.write(json.dumps(all_dict_list))

1913989

In [ ]:
crawling_df = pd.read_json(file_path+"result_dict(0825).json")
crawling_df = crawling_df[['p_code','cr','name','link','price','cate1','cate2','cate3']]
crawling_df.to_csv(file_path+"crawl_uniquecode(0825).csv", index = False)

In [ ]:
crawling_df[['cr']]

,p_code,cr,name,link,price,cate1,cate2,cate3
0,201072,1,TATE 남성 캐시미어 블랜딩 니트3종,https://cr.shopping.naver.com/adcr.nhn?x=9RVsF...,68510원,패션의류,남성의류,니트/스웨터
1,201079,1,테이트 여성 19FW 헤어리 폭스블렌디드 니트 3종,https://cr.shopping.naver.com/adcr.nhn?x=LiUk2...,39110원,패션의류,여성의류,니트/스웨터
2,200974,0,None,None,None,None,None,None
3,202377,1,[신세계TV쇼핑]Cerini by PAT 남성 소프트기모 릴랙스 팬츠 3종,https://cr.shopping.naver.com/adcr.nhn?x=IZE1J...,56910원,스포츠/레저,골프,골프의류
4,202404,0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
2119,201819,1,[레스토닉]유로탑 블루투스 LED서랍형 침대SS,https://cr.shopping.naver.com/adcr.nhn?x=NmMW7...,664050원,가구/인테리어,침실가구,침대
2120,201821,1,[현대홈쇼핑]레스토닉 [레스토닉] 유로탑 블루투스 LED 서랍형침대 Q,https://cr.shopping.naver.com/adcr.nhn?x=vEYSS...,519910원,가구/인테리어,침실가구,침대
2121,201823,0,None,None,None,None,None,None
2122,200196,0,None,None,None,None,None,None


## 2. 크롤링 안된 상품명들 확인 

In [ ]:
# get p name of not cr data 
not_cr_data = cr_data[cr_data['cr'] != 1]
not_cr_data['p_name'] = not_cr_data['p_code'].apply(code2name)
not_cr_data

,p_code,cr,name,link,price,cate1,cate2,cate3,p_name
2,200974,0,NaN,NaN,NaN,NaN,NaN,NaN,오모떼 레이스 파운데이션 브라
4,202404,0,NaN,NaN,NaN,NaN,NaN,NaN,보코 리버시블 무스탕
8,200970,0,NaN,NaN,NaN,NaN,NaN,NaN,오모떼 360도 텐션업 레이스 패키지 시즌4
9,201220,0,NaN,NaN,NaN,NaN,NaN,NaN,그렉노먼 남성 구스다운 롱 벤치코트
10,201226,0,NaN,NaN,NaN,NaN,NaN,NaN,그렉노먼 여성 구스다운 롱 벤치코트
...,...,...,...,...,...,...,...,...,...
2111,201264,0,NaN,NaN,NaN,NaN,NaN,NaN,[RYN] 린 여성 다이얼락 히트 웜 퍼 방한화 2종
2117,201357,0,NaN,NaN,NaN,NaN,NaN,NaN,루시헨느 레이스 홀리데이 란쥬 패키지
2121,201823,0,NaN,NaN,NaN,NaN,NaN,NaN,레스토닉 블루투스 LED 서랍 침대 킹
2122,200196,0,NaN,NaN,NaN,NaN,NaN,NaN,삼성화재 행복한파트너 주택화재보험(1912)
